In [1]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
#train set
df_data = pd.read_csv('b_depressed.csv')

# #show the shape of the train dataframe
df_data.shape

(1429, 23)

In [3]:
 df_data.head()

,Survey_id,Ville_id,sex,Age,Married,Number_children,education_level,total_members,gained_asset,durable_asset,...,incoming_salary,incoming_own_farm,incoming_business,incoming_no_business,incoming_agricultural,farm_expenses,labor_primary,lasting_investment,no_lasting_investmen,depressed
0,926,91,1,28,1,4,10,5,28912201,22861940,...,0,0,0,0,30028818,31363432,0,28411718,28292707.0,0
1,747,57,1,23,1,3,8,5,28912201,22861940,...,0,0,0,0,30028818,31363432,0,28411718,28292707.0,1
2,1190,115,1,22,1,3,9,5,28912201,22861940,...,0,0,0,0,30028818,31363432,0,28411718,28292707.0,0
3,1065,97,1,27,1,2,10,4,52667108,19698904,...,0,1,0,1,22288055,18751329,0,7781123,69219765.0,0
4,806,42,0,59,0,4,10,6,82606287,17352654,...,1,0,0,0,53384566,20731006,1,20100562,43419447.0,0


In [4]:
df_data.drop(['Survey_id','Ville_id'],axis=1,inplace=True)
df_data.drop(['gained_asset','durable_asset','save_asset'],axis=1,inplace=True)
df_data.drop(['other_expenses','incoming_own_farm','incoming_no_business','incoming_agricultural','farm_expenses'],axis=1,inplace=True)

In [5]:
# split data into train and test sets
X = df_data.drop(["depressed"], axis=1)

from sklearn.impute import SimpleImputer
# fill missing values with mean column values
imputer = SimpleImputer()
transformed_X = imputer.fit_transform(X)

y = df_data.depressed

seed = 5
test_size = 0.33
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=test_size, random_state=seed)

In [7]:
# create model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
model = Sequential()
model.add(Dense(36, input_dim=12, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [8]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# Fit the model
model.fit(X_train, y_train, epochs = 100, batch_size =10)

Epoch 1/100
96/96 [==============================] - 1s 2ms/step - loss: 2701007.4034 - accuracy: 0.4201
Epoch 2/100
96/96 [==============================] - 0s 2ms/step - loss: 91321.2284 - accuracy: 0.6706
Epoch 3/100
96/96 [==============================] - 0s 2ms/step - loss: 42211.5757 - accuracy: 0.6923
Epoch 4/100
96/96 [==============================] - 0s 2ms/step - loss: 50735.5174 - accuracy: 0.7198
Epoch 5/100
96/96 [==============================] - 0s 2ms/step - loss: 35730.6326 - accuracy: 0.6835
Epoch 6/100
96/96 [==============================] - 0s 2ms/step - loss: 43300.2781 - accuracy: 0.6689
Epoch 7/100
96/96 [==============================] - 0s 2ms/step - loss: 27790.2469 - accuracy: 0.6990
Epoch 8/100
96/96 [==============================] - 0s 2ms/step - loss: 31313.2198 - accuracy: 0.6666
Epoch 9/100
96/96 [==============================] - 0s 2ms/step - loss: 23013.9246 - accuracy: 0.7153
Epoch 10/100
96/96 [==============================] - 0s 2ms/step - los

96/96 [==============================] - 0s 3ms/step - loss: 6466.3142 - accuracy: 0.6993
Epoch 80/100
96/96 [==============================] - 0s 3ms/step - loss: 5267.0332 - accuracy: 0.7309
Epoch 81/100
96/96 [==============================] - 0s 3ms/step - loss: 8417.0251 - accuracy: 0.6921
Epoch 82/100
96/96 [==============================] - 0s 4ms/step - loss: 6946.4153 - accuracy: 0.7210
Epoch 83/100
96/96 [==============================] - 0s 3ms/step - loss: 12127.4720 - accuracy: 0.6651
Epoch 84/100
96/96 [==============================] - 0s 3ms/step - loss: 7748.0204 - accuracy: 0.6916
Epoch 85/100
96/96 [==============================] - 0s 4ms/step - loss: 6856.6073 - accuracy: 0.7302
Epoch 86/100
96/96 [==============================] - 0s 4ms/step - loss: 7423.5348 - accuracy: 0.7335
Epoch 87/100
96/96 [==============================] - 0s 3ms/step - loss: 8517.7786 - accuracy: 0.7276
Epoch 88/100
96/96 [==============================] - 0s 3ms/step - loss: 7138.8578 -

In [10]:

scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

15/15 [==============================] - 0s 2ms/step - loss: 5116.7476 - accuracy: 0.7627

accuracy: 76.27%


In [11]:
keras_file = "DEPRESSION26.h5"
from tensorflow import keras
keras.models.save_model(model, keras_file)

In [17]:
import tensorflow as tf
# converter = tf.contrib.lite.TocoConverter.from_keras_model_file(keras_file)
# tflite_model = converter.convert()
# open("DEPRESSION26.tflite","wb").write(tflite_model)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\namde\AppData\Local\Temp\tmptkqu98v5\assets


INFO:tensorflow:Assets written to: C:\Users\namde\AppData\Local\Temp\tmptkqu98v5\assets


9088